In [ ]:
import numpy as np
from numpy.fft import *
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import os.path as path
import scipy as sp
import scipy.fftpack
from scipy import signal
from pykalman import KalmanFilter
from sklearn import tree
import tensorflow as tf
import tensorflow_hub as hub

import gc

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


DATA_PATH = "../input/liverpool-ion-switching"

x = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
#test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
#submission_df = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

In [ ]:
# reference: http://kitchingroup.cheme.cmu.edu/blog/2013/02/03/Using-Lagrange-multipliers-in-optimization/
# Calculate Hamiltonian
def func(X):
    x = X[0]
    y = X[1]
    L = X[2] 
    return x + y + L * (x**2 + k * y)

# dirvitive of Hamiltionian with respect to x
def dfunc(X):
    dL = np.zeros(len(X))
    d = 1e-4 
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = d
        dL[i] = (func(X+dX)-func(X-dX))/(2*d);
    return dL

In [ ]:
def ion_plotter(x, examples):

    fig, ax = plt.subplots(nrows=len(examples), ncols=1, figsize=(25, 3.5*len(examples)))
    fig.subplots_adjust(hspace = .5)
    ax = ax.ravel()
    colors = plt.rcParams["axes.prop_cycle"]()

    for i in range(len(examples)):
    
        c = next(colors)["color"]
        ax[i].grid()
        if examples[i] in ['signal_f','signal','message_current','injected_current']:
            ax[i].plot(x['time'], x[examples[i]],color=c, linewidth= 2)
            ax[i].set_ylabel('current (pA)', fontsize=14)
        if examples[i] in ['y_var','y_var_est','y_var_k']:
            ax[i].scatter(x['time'][::100], x[examples[i]][::100],marker ='.', color=c, linewidth=0)
            ax[i].set_ylabel('Open Channels', fontsize=14)
            ax[i].set_ylim(-5,5)
        if examples[i] in ['y_mean_est','y_mean','y_pred','y_mean_k']:
            ax[i].scatter(x['time'][::100], (x[examples[i]][::100]),marker ='.', color=c, linewidth=0)
            ax[i].set_ylabel('Open Channels', fontsize=14)
            ax[i].set_ylim(0,10)
        if examples[i] in ['y','y_est','y_est_k']:
            ax[i].scatter(x['time'][::100], (x[examples[i]][::100]),marker ='.', color=c, linewidth=0)
            ax[i].set_ylabel('Open Channels', fontsize=14)
            ax[i].set_ylim(0,10)
        if examples[i] in ['signal_energy','injected_energy','message_energy']:
            ax[i].plot(x['time'], x[examples[i]],color=c, linewidth= 1)
            ax[i].set_ylabel('Energy 10^-24 Joules', fontsize=14)                     
        ax[i].plot(x['time'], x[examples[i]],color=c, linewidth=.5)
        ax[i].set_title(examples[i], fontsize=24)
        ax[i].set_xlabel('Time (seconds)', fontsize=14)
        #ax[i].set_ylabel('current (pA)', fontsize=24)
        #ax[i].set_ylim(0,5)

In [ ]:
x = x.rename(columns = {'open_channels':'y'})

# Find quiet_probe_current  when y = 0
x['qs'] = x['signal'] - x['signal'].shift(1)
x.loc[0,'qs'] = 0.
qsm = x.loc[x.y == 0.,['qs']].values
qq = np.mean(qsm) 

print(f'{qq:.3f} pA = Quiet_probe_current')



## Reduce known non-gaussian noise.

To eliminate low frequency waveform noise, the filter should block DC and VLF. In train data from 365s to 385s exists obvious 100Hz and harmonics of 100Hz (n x 100Hz).

Tho remove both of these noises I'll first try a Comb filter using it's difference equation:

x<sub>1</sub>(t) = a0 x<sub>0</sub>(t) + a1 x<sub>0</sub>(t-K)<br>
K = fs/f0<br>
a0 = 1<br>
a1 = -0.99

In [ ]:


x['y'] = x['y'].astype('int')
dt = 0.0001
fs = 10000
f0 = 100

K = np.int(fs/f0)
a1 = -0.99 
x['signal_f'] = 0.
x['signal_f'] = x['signal'] + a1 * x['signal'].shift(K)
x.loc[0:K-1,'signal_f'] = x.loc[0:K-1,'signal']

filter_gain =  np.sqrt( (x['signal']**2).sum()/(x['signal_f']**2).sum()) 
print(f'filter gain is {filter_gain:.3f}')
print(f'filter loss is {1./filter_gain:.3f}')


## Calculate variance: y'(t) and mean: Expected{y(t)}
y'(t) = 1/k  r2/r signal_current(t)

The injection current raises the mean of y. y_mean(t) = - 1/k * insertion_current(t) 

## Injection energy is converted into open channel Potential Energy y'(t) * k

x2 above, the injection energy (IE) of current IC, frees ion channels to open with more vigor. When IE is lowest (0.1 x10^-24 Joules), it is rare for more than 1 channel to be open. As IE increase, so does ion channel freedom to open. At IE = 6.0 x10^-24 J, we see up to 10 open channels.

Other notebooks have shown that given the mode {0,1,2,3,4}, the state of open channels are gaussian distributions. We also make the assumption that after initial comb filtering (Drift removal) that our signal also has a gaussian distribution. I assume that the injection current is DC and has variance=0. Gaussians have continuous first and second derivatives (a HJB requirement). This also inspires me to try to imagine the "True" reciever's transfer function H(z) as a gaussian and as the sum of two gaussians:

     gaussian(y (mean,var)) = gaussian(injection_energy(mean,var=0)) + gaussian(message_energy(mean,var))
     y(mean) = injection_energy_mean + message_energy_mean
     y(var) = message_energy_variance

In [ ]:
r = 70000 # Ohms
r2 = 50 # Ohms
r1 = 50 # Ohms
k = 0.0016 # (pC/ion channel change)

# First the true variance of y
x['y_var'] = x['y'] - x['y'].shift(1)
x.loc[0,'y_var'] = 0
x['y_var'] = x['y_var'].astype('int')
x['y_mean'] = x['y'].rolling(window=100, min_periods=5).mean()
x.loc[0:4,'y_mean'] = 0


x['y_var_est'] = r2 * x['signal_f'] / (k * r)
x['y_var_est'] = x['y_var_est'].round(0).clip(-10,10).astype('int')

# the injection current can be estimated as the most negative, min() of the signal

x['injected_current'] = x['signal_f'].rolling(window=7500,min_periods=5).min()
x.loc[0:4,'injected_current'] = 0.

x['injected_energy'] = r1 * x['injected_current']**2 * dt/32 # I think injected current is square wave

x['y_mean_est'] =  1/k * x['injected_energy']
x.loc[0:4,'y_mean_est'] = 0

# And now estimate y(t) = y_var(t) + y_mean(t)

x['y_est'] = x['y_mean_est'] + x['y_var_est']
x['y_est'] = x['y_est'].round(0).clip(0,10).astype('int')

examples = ['signal_f', 'y_var', 'y_var_est', 'y_mean','y_mean_est','y','y_est']
ion_plotter(x,examples)

In [ ]:
plt.close
# Calculate the f1_score for the estimate
f1 = f1_score(x.y, x.y_est, average='macro')
print(f'f1_score is {f1:.3f}')

In [ ]:
# Reference 3
from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: '𝒩(μ={:.3f}, 𝜎²={:.3f})'.format(s[0], s[1])

def update(prior, measurement):
    x, P = prior        # mean and variance of prior of x (system)
    z, R = measurement  # mean and variance of measurement (open_channels) with ion probe
    
    J = z-x          #1 - f1_score(z,x)        # residual - This is error we want to minumize
    K = P / (P + R)              # Kalman gain

    x = x + K*J      # posterior
    P = (1 - K) * P  # posterior variance
    return gaussian(x, P)

def predict(posterior, movement):
    x, P = posterior # mean and variance of posterior
    dx, Q = movement # mean and variance of movement
    x = x + dx
    P = P + Q
    return gaussian(x, P)


In [ ]:
# set initial conditions
prior = gaussian(1,1)


for i in range(len(x)):
    measurement = gaussian(x.loc[i,'y_mean_est'],x.loc[i,'y_var_est'])        
    
    post = update(prior,measurement)
    
    movement = gaussian(x.loc[i,'y_var'],x.loc[i,'y_mean'])
    est, P = predict(post,movement)
    x.loc[i,'y_mean_k'] = est
    x.loc[i,'y_var_k'] = P

x['y_est_k'] = x['y_mean_k'] + x['y_var_k']
x['y_est_k'] = x['y_est_k'].round(0).clip(0,10).astype('int')
examples = ['y','y_est','y_est_k']
ion_plotter(x,examples)

In [ ]:
# Calculate the f1_score for the estimate
# first descretize the estimate by rounding to ones digit.

f1 = f1_score(x.y, x.y_est, average='macro')
print(f'starting f1_score is {f1:.3f}')
f1 = f1_score(x.y, x.y_pred_k, average='macro')
print(f'f1_score after optimization is {f1:.3f}')

In [ ]:
from itertools import islice

def window(seq, n=2):
    "Sliding window width n from seq.  From old itertools recipes."""
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result
        
pairs = pd.DataFrame(window(x.loc[:,'y']), columns=['state1', 'state2'])
counts = pairs.groupby('state1')['state2'].value_counts()
alpha = 1 # Laplacian smoothing is when alpha=1
counts = counts + 1
#counts = counts.fillna(0)
P = ((counts + alpha )/(counts.sum()+alpha)).unstack()
P

In [ ]:

pairs = pd.DataFrame(window(x.loc[:,'signal_energy']), columns=['state1', 'state2'])
means = pairs.groupby('state1')['state2'].mean()
alpha = 1 # Laplacian smoothing is when alpha=1
means = means.unstack()
means

In [ ]:
print('Occurence Table of State Transitions')
ot = counts.unstack().fillna(0)
ot

In [ ]:
P = (ot)/(ot.sum())
Cal = - P * np.log(P)
Cal

In [ ]:
Caliber = Cal.sum().sum()
sns.heatmap(
    Cal,
    annot=True, fmt='.3f', cmap='Blues', cbar=False,
    ax=axes, vmin=0, vmax=0.5, linewidths=2);

In [ ]:
# reference https://www.kaggle.com/friedchips/on-markov-chains-and-the-competition-data
def create_axes_grid(numplots_x, numplots_y, plotsize_x=6, plotsize_y=3):
    fig, axes = plt.subplots(numplots_y, numplots_x)
    fig.set_size_inches(plotsize_x * numplots_x, plotsize_y * numplots_y)
    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    return fig, axes

fig, axes = create_axes_grid(1,1,10,5)
axes.set_title('Markov Transition Matrix P for all of train')
sns.heatmap(
    P,
    annot=True, fmt='.3f', cmap='Blues', cbar=False,
    ax=axes, vmin=0, vmax=0.5, linewidths=2);

In [ ]:
eig_values, eig_vectors = np.linalg.eig(np.transpose(P))
print("Eigenvalues :", eig_values)

Lagrangian analysis seeks to find minimums and maxima for prediction purposes. First find the Lagrangian L such that:

f(x,y) = L g(x,y)


In [ ]:
from scipy.optimize import fsolve

# this is the max
X1 = fsolve(dfunc, [1, 1, 0])
print(X1, func(X1))

# this is the min
X2 = fsolve(dfunc, [-1, -1, 0])
print(X2, func(X2))

In [ ]:
del x